In [1]:
# CELL A
# YOU MUST ALWAYS RUN THIS CELL BEFORE
# RUNNING ANY OTHERS
# If you run all the cells in order, they will work.
# Once you have produced and summarized some simulation
# results using cells B-G
# You could just run this cell and then skip to cell H
# to play around wit plotting
import os
from combined_theory import get_C, get_variance_units_little_del_square

# NBNBNB CHANGE my_base_directory_local
# to be the directory in which you place the folder
# called "code"
my_base_directory_local = "directory_contain_poly1D/PolygenicAdaptation1D"

my_base_directory_local = '/Users/Alexandra/pcloud_synced/PycharmProjects/Polished_code_python3/PolygenicAdaptation1D'

save_output_dir =  os.path.join(my_base_directory_local,'results_local_test_output')
save_sims_directory = os.path.join(my_base_directory_local,'results_local_test')
save_sims_land_directory =  os.path.join(my_base_directory_local,'results_local_test_lande')
code_directory = os.path.join(my_base_directory_local,"code")

# the population size is 1000
N = 1000
# the shift size in units of initial phenotypic SD is 2
D_s0 = 2
# the number of mutations per gamete per gen
U = 0.02
# expected scaled selection coeff of new muts
# (squared effects have a gamma distribution)
E2Ns = 10
# If you don't choose the variance in scaled
#selection coefficient it defaults to E2Ns^2
# making the distribution of incoming squared
#effects exponential

sigma_0_del = get_variance_units_little_del_square(E2Ns,mut_input=2*N*U)**(0.5)
C = get_C(E2Ns)
shift_over_root_Vs = D_s0*sigma_0_del/float(2*N)**(0.5)
print("C, the allelic measure of deviation from Lande, for your chosen squared\n"
      " effect size distribution is: "+str(C))
print("\n")
print("The steady-state standard deviation in phenotype distribution for your chosen\n"
      "parameters in units little delta = root(Vs/(2N)) is: " + str(sigma_0_del))
print("To be in the parameter regime considered in the paper, you want it to be >>1,\n"
      "and << root(2N)= "+ str((2.0*N)**(0.5)))
print("\n")
print("The shift/root(Vs) is: " + str(shift_over_root_Vs))
print("To be in the parameter regime considered in the paper, "
      "\nyou want it to on the order of 1 or smaller")
print("\n")
print("The simulations (AA and OA) that run using an approximation for E[dx] (which\n"
      "assumes that integrals of certain functions over the phenotype distribution\n"
      "can be approximated by the function value at the mean phenotype) are not\n"
      "valid unless 1 >> VA/Vs, which at steady-state for your parameter\n"
      "choices is " + str(sigma_0_del**2/float(2*N)))
print("ALL the trajectory simulations (OA) use this assumption\n"
      "as do the population simulations just follow alleles (AA).")
print("Thus if this condition is not satisfied, ONLY the population simulattions\n"
      "that simulate the FULL MODEL will be accurate (don't use the others) ")

# save the params in a dict
# so that later we can locate the folder
# with the simulation results, and run OA
# trajectory simulations using the D(t)
# from those results
param_dict = dict()
param_dict['N'] = N
param_dict['U'] = U
param_dict['shift_s0'] = D_s0
param_dict['E2Ns'] = E2Ns

C, the allelic measure of deviation from Lande, for your chosen squared
 effect size distribution is: 0.6840765649154079


The steady-state standard deviation in phenotype distribution for your chosen
parameters in units little delta = root(Vs/(2N)) is: 12.73393504719654
To be in the parameter regime considered in the paper, you want it to be >>1,
and << root(2N)= 44.721359549995796


The shift/root(Vs) is: 0.5694788877319691
To be in the parameter regime considered in the paper, 
you want it to on the order of 1 or smaller


The simulations (AA and OA) that run using an approximation for E[dx] (which
assumes that integrals of certain functions over the phenotype distribution
can be approximated by the function value at the mean phenotype) are not
valid unless 1 >> VA/Vs, which at steady-state for your parameter
choices is 0.08107655089311018
ALL the trajectory simulations (OA) use this assumption
as do the population simulations just follow alleles (AA).
Thus if this condition is not 

In [2]:
#CELL B
# This cell will take about 10-15 min to
# run for the params chosen in Cell A.
# Recommend running this >=5 times
# to get reasonable plots
# And many, many more times if you want the trajectory sims (OA)
# that use the average D(t) from these sims to be accurate

# This cell simulates the populations
# The default parameters are chosen, so that the sims
# will not take long to test on your computer
# In particular you will want a better
# realistic burn time, lag time, population size
# and mutation rate
# Maybe about 10 min for the params chosen in Cell A

code_file = os.path.join(code_directory,"simulate_populations_argparse.py")

# the population size is N
arguments = " -N "+str(param_dict['N'])
# the shift size in units of initial phenotypic SD is D_s0
arguments += " -D_s0 " + str(param_dict['shift_s0'])
# the number of mutations per gamete per gen is U
arguments += " -U " + str(param_dict['U'])
# expected scaled selection coeff of new muts is E2Ns
arguments += " -E2Ns "+str(param_dict['E2Ns'])

# the -hi flag means that histogram stats are saved (recommend)
# the -r 2 means that the simulator will do two runs
# of evolution
# I set it to 2 rather then 1, so you will have
# something to test the code that summarizes the results of
# multiple runs of evolution
# If there is only one run, there is nothing
# to summarize.
# In reality you will want to simulate many more than
# 3 runs of evolution
arguments += " -hi -r 2 -sD " + save_sims_directory
# arguments += " -a exact "
run_in_terminal = "python " + code_file + arguments

# saves the standard error and standard output to text files,
# which you can look at if something goes wrong
run_in_terminal += ' > stdout.txt 2> stderror.txt'

# Running the code via the terminal
os.system(run_in_terminal)


0

In [3]:
#CELL C
# Run this before the next cell
# Text results wil be saved in
# results_local_test_output/text_output
# inside your chosen my_base_directory_local

# This cell summarizes the results and saves the summaries
# using summarize_save_argparse.py
# It also deletes the individual runs. If you have run time-consuming
# simulations with the full population, as opposed to the All Alleles version,
# you may not want to delete the individual runs
# This also saves some results to a text file

code_file = os.path.join(code_directory,"summarize_save_argparse.py")
arguments = " -rD "+ save_sims_directory + " -sD " + save_output_dir
run_in_terminal = "python " + code_file + arguments

# saves the standard error and standard output to text files,
# which you can look at if something goes wrong
run_in_terminal += ' > stdout.txt 2> stderror.txt'

# Running the code via the terminal
os.system(run_in_terminal)

0

In [6]:
#CELL D
#YOU must first have run Cell B Cell C
# After you run this cell, you must run Cell G before plotting the results
# from this cell

# This cell will take ~10 min for the parameters
# chosen in Cell A

# This cell simulates allele trajectories using D(t) from
# population simulations
# The default parameters are silly, so that the sims
# will not take too long to test on your computer

# if Lande is true, the the sims will run with the Lande version of D(t),
print("NOTE: YOU WILL NOT GET ACCURATE RESULTS WHEN LANDE = FALSE, UNLESS YOU HAVE RUN (and summarized)"
      " ENOUGH FULL POPULATION SIMS\n TO HAVE A NOT TOO LARGE STANDARD ERROR IN D(T)\n(or mu_3(t)/2V_A (t) "
      "which we use as a proxy for D(t) in the equilibration period)")
lande = False

# Let's run the program a few times so there is something
# to summarize
num_runs = 4

# save_sims_directory needs to contain sims corresponding to an
# actual simulation folder with population simulation results
# This is because (if lande=False) we are reading D(t) from
# these simulations
# If Lande is True, we are reading the phenotypic variance
# and the shift size from these simulations, but using the
# the corresponding Lande D(t)

code_file = os.path.join(code_directory,"simulate_trajectories_argparse.py")

arguments = " -psF " + save_sims_directory
# the -nM 250 means that each tuple has 10 alleles
# In reality you would want to simulate many more than 250 alleles for
# each, say, effect size to get reasonable estimates of statistics
# corresponding to allele trajectories of alleles with that effect.
# I just chose parameters so the sims will run not too slowly for testing
arguments += " -nM 250 "

# if it exists, the program will find the simulation folder,
# in psF=save_sims_directory for the population simulations
# that you previously ran. It will read D(t)
# from those sims and simulate trajectories using that
# D(t). In the Lande case, it will calculate sigma_0_del
# for those parameters and run simulations with the
# corresponding Lande D(t)

# the population size is N
arguments += " -N "+str(param_dict['N'])
# the shift size in units of initial phenotypic SD is D_s0
arguments += " -D_s0 " + str(param_dict['shift_s0'])
# the number of mutations per gamete per gen is U
arguments += " -U " + str(param_dict['U'])
# expected scaled selection coeff of new muts is E2Ns
arguments += " -E2Ns "+str(param_dict['E2Ns'])
if lande:
    arguments+= ' -l'
run_in_terminal = "python " + code_file + arguments

# saves the standard error and standard output to text files,
# which you can look at if something goes wrong
run_in_terminal += ' > stdout.txt 2> stderror.txt'

# Running the code num_runs times via the terminal
for _ in range(num_runs):
    os.system(run_in_terminal)

print("You need to simulate a fairly large number of alleles to get reasonable results. In the paper we did 250 000.\n"
  "Note that when you simulate too few alleles, the 1.96*se (standard error) will NOT provide good confidence\nintervals "
  "as the central limit does not apply")


You need to simulate a fairly large number of alleles to get reasonable results. In the paper we did 250 000.
Note that when you simulate too few alleles, the 1.96*se (standard error) will NOT provide good confidence
intervals as the central limit does not apply
NOTE: YOU WILL NOT GET ACCURATE RESULTS WHEN LANDE = FALSE, UNLESS YOU HAVE RUN (and summarized) ENOUGH FULL POPULATION SIMS
 TO HAVE A NOT TOO LARGE STANDARD ERROR IN D(T)
(or mu_3(t)/2V_A (t) which we use as a proxy for D(t) in the equilibration period)
You need to simulate a fairly large number of alleles to get reasonable results. In the paper we did 250 000.
Note that when you simulate too few alleles, the 1.96*se (standard error) will NOT provide good confidence
intervals as the central limit does not apply


In [7]:
#Cell E
# Setting the parameters for the PURE LANDE trajectoories (OA)
# (in Cell F)
# For these trajectories you don't need a preexisting simulation folder with simulation results, from which

# the steady state phenotypic standard deviation is 20 in units little delta
sigma_0_del = 20
# the population size is 5000
N = 5000
# the shift size in units of initial phenotypic SD is 2
D_s0 = 2

shift_over_root_Vs = D_s0*sigma_0_del/float(2*N)**(0.5)
print("Your chosen steady-state standard deviation in phenotype distribution\n"
      "in units little delta = root(Vs/(2N)) is: " + str(sigma_0_del))
print("To be in the parameter regime considered in the paper, you want it to be >>1,\n"
      "and << root(2N)= "+ str((2.0*N)**(0.5)))
print("\n")
print("The shift/root(Vs) is: " + str(shift_over_root_Vs))
print("To be in the parameter regime considered in the paper, "
      "\nyou want it to on the order of 1 or smaller")
print("\n")
print("The trajectory simulations (OA) run using an approximation for E[dx] (which\n"
      "assumes that integrals of certain functions over the phenotype distribution\n"
      "can be approximated by the function value at the mean phenotype) are not\n"
      "valid unless 1 >> VA/Vs, which at steady-state for your parameter\n"
      "choices is " + str(sigma_0_del**2/float(2*N)))
print("Thus if this condition is not satisfied, do not run trajectory simulations.\n"
      "As the results will not be valid ")

Your chosen steady-state standard deviation in phenotype distribution
in units little delta = root(Vs/(2N)) is: 20
To be in the parameter regime considered in the paper, you want it to be >>1,
and << root(2N)= 100.0


The shift/root(Vs) is: 0.4
To be in the parameter regime considered in the paper, 
you want it to on the order of 1 or smaller


The trajectory simulations (OA) run using an approximation for E[dx] (which
assumes that integrals of certain functions over the phenotype distribution
can be approximated by the function value at the mean phenotype) are not
valid unless 1 >> VA/Vs, which at steady-state for your parameter
choices is 0.04
Thus if this condition is not satisfied, do not run trajectory simulations.
As the results will not be valid 


In [8]:
# Cell F
# Should take ~20 min with parameters chosen in cell E
# First run cell E which specifies your parameter choices
# You must run Cell G before plotting the results
# from this one

# This cell simulates the PURE Lande trajectories. So unlike the previous run of trajectory
# simulations you don't need a preexisting simulation folder with simulation results, from which
# the simulator reads parameters (or possibly also D(t)).
# Instead, you specify a population size, shift size and steady-state
# phenotypic standard deviation (in units little delta(=sqrt(Vs/2N))).
# The simulator then simulates trajectories of alleles with the
# corresponding Lande D(t)
# The default parameters (specifically the nM the number of mutations)
# are chosen so that the sims will not take long to test on your computer

# Let's run the program a few times so there is something
# to summarize
num_runs = 5

code_file = os.path.join(code_directory,"simulate_trajectories_argparse.py")

# the steady state phenotypic standard deviation is sigma_0_del in units little delta
arguments = " -sigma_0_del "+str(sigma_0_del)
# the population size is N
arguments += " -N "+str(N)
# the shift size in units of initial phenotypic SD is D_s0
arguments += " -D_s0 "+str(D_s0)
# as beforethe -nM 100 means that each tuple has 100 mutations
# of evolution

# ONLY SIMULATING with 100 mutatins per effect
# size. You definitely want to use more than this
arguments +=  ' -nM 100'
# It is '-oL' that indicates we are running PURE Lande
arguments += " -oL"
# specify the folder name to save pure Lande traj results
arguments += " -lF " + save_sims_land_directory
run_in_terminal = "python " + code_file + arguments

# saves the standard error and standard output to text files,
# which you can look at if something goes wrong
run_in_terminal += ' > stdout.txt 2> stderror.txt'

# Running the code num_runs times via the terminal
for _ in range(num_runs):
    os.system(run_in_terminal)

In [3]:
# Cell G
# You must run this cell after you have run
# either Cell D or Cell E&F, to summarize
# those results
# Text results wil be saved in
# results_local_test_output/text_output
# inside your chosen my_base_directory_local

# This cell summarizes the results and saves the summaries
# Also deletes the individual runs
# Also saves some results to a text file

# Lande is true if you are summarising pure Lande trajectory
# results
# Otherwise lande=False
lande_list =[True,False]

code_file = os.path.join(code_directory,"summarize_save_argparse.py")
arguments = " -rD "+ save_sims_directory + " -sD " + save_output_dir
arguments_Lande = " -rD "+ save_sims_land_directory + " -sD " + save_output_dir
run_in_terminal = "python " + code_file + arguments
run_in_terminal_Lande = "python " + code_file + arguments_Lande

# Uncomment below if you need to see the output and/or standard error
run_in_terminal += ' > redirected.txt 2> stderror.txt'
for lande in lande_list:
    if not lande:
        os.system(run_in_terminal)
    else:
        os.system(run_in_terminal_Lande)

In [2]:
#Cell H
###PLOTTING SETUP#######
# Always run this cell before running any of the
# cells below to plot figures.
# Figures will be saved in
# results_local_test_output/figures
# inside your chosen my_base_directory_local.
import os
from summarize_save_class import SumAndSave

# This cell just sets us up to plot a few very rough figures
# whic we do in the following two cells

save_figure_directory = os.path.join(save_output_dir, "figures")

# parent directory is the directory were you results directories are stored
# save_output_dir is the directory you want new output to be saved
#You must have run population sims, with cell B
# and summarized them with Cell C first
my_sum_and_save= SumAndSave(parent_dir=save_sims_directory,save_output_dir=save_output_dir)

#You must have run pure lande trajectory sims, with Cell E&F
# and summarized them with Cell G first
my_sum_and_save_Land = SumAndSave(parent_dir=save_sims_land_directory,save_output_dir=save_output_dir)


{'N': 1000, 'Vs': 2000.0, 'shift_s0': 2.0, 'var_0': 162.15310178621888, 'sigma_0_del': 12.733935047196482, 'shift': 25.467870094392964, 'U': 0.02, 'E2Ns': 10.0, 'V2Ns': 100.0, 'scale_s': 10.0, 'shape_s': 1.0, 'lag_time': 1000, 'T_max': 10901, 'algorithm': 'approx', 't_freeze_new_muts_start': 0, 't_length_freeze_new_muts': 5000, 't_freeze': 0, 'burn_time': 5000}
{'N': 1000, 'Vs': 2000.0, 'shift_s0': 2.0, 'var_0': 162.15310178621888, 'sigma_0_del': 12.733935047196482, 'shift': 25.467870094392964, 'U': 0.02, 'E2Ns': 10.0, 'V2Ns': 100.0, 'scale_s': 10.0, 'shape_s': 1.0, 'lag_time': 1000, 'T_max': 10901, 'algorithm': 'approx', 't_freeze_new_muts_start': 0, 't_length_freeze_new_muts': 5000, 't_freeze': 0, 'burn_time': 5000, 'var_0_emp': 173.31332001788564, 'var_0_emp_se': 4.73427527075523}
final_stats  not in traj stats
reading param dict
{'N': 5000, 'Vs': 10000.0, 'shift_s0': 2.0, 'var_0': 400.0, 'sigma_0_del': 20.0, 'shift': 40.0, 'time_from_which_use_dist_guess': 0, 'var_0_emp': 400.0}
fi

In [4]:
#Cell I
# First run Cell H to set up sum_and_saves for plotting
#You also must have run population sims, with cell B
# and summarized them with Cell C

# The code that does the plotting is messy and hard to understand
# You can also use the text files to generate whatever plots you wish.
# This cell allows you to quickly look at some results from the
# full population simulations

# This cell plots some basic statistics like the
# distance of the mean phenotype from the optimum (dist)
# or the skewness or variance of the phenotype distribution
# dist_guess = third moment/(2*variance) of
# the phenotype distribution

# The saved plots appear in results_local_test_output/figures/approx/parameter_folder
# 'approx' indicates that the approximate population sims (AA) were used
# to generate the sim results, and 'parameter_folder' has the names of
# the various paraters used in the text

from plot_class import PlotClass


bstat_list = ['skewness','opt_minus_mean_fixed_over_shift','dist_guess']
bstat_list += ['d2ax_frozen_over_shift','dist_sum_over_2N_scaled']

#better to look at these stats with trait in units
# of initial phenotypic SD
bstat_list_unts_s0 = ['var','dist']

end_time = 5000
first_time = 0

for data_class_param_dict in my_sum_and_save.param_dict_list():
    myparamdictkey = my_sum_and_save.turn_dict_into_dict_key(data_class_param_dict)
    datafolder = my_sum_and_save.make_folder_name_for_data_class(myparamdictkey)
    current_save_figure_directory = os.path.join(save_figure_directory,datafolder)
    tmp_dclass = my_sum_and_save.get_data_class(myparamdictkey)
    parameter_string = tmp_dclass.make_parameter_text_to_write_file()
    current_save_figure_directory = os.path.join(current_save_figure_directory,parameter_string)
    if not os.path.exists(current_save_figure_directory):
        os.makedirs(current_save_figure_directory)

    # if units are true we plot
    # in units of steady-state SD
    # if units are false we plot
    # in units of little delta (=sqrt(Vs/2N))
    tmp_dclass.set_units(False)

    tmp_plots = PlotClass([tmp_dclass],base_dir=current_save_figure_directory)

    for bstati in bstat_list:
        tmp_plots.plot_bstat(bstat=bstati,std=False,domain=[first_time ,end_time])

    tmp_dclass.set_units(True)
    tmp_plots = PlotClass([tmp_dclass],base_dir=current_save_figure_directory)
    for bstati in bstat_list_unts_s0:
        tmp_plots.plot_bstat(bstat=bstati,std=False,domain=[first_time ,end_time])
        tmp_dclass.set_units(True)


In [5]:
# Cell J
# First run Cell H to set up sum_and_saves for plotting
# You also must have run population sims, with cell B
# and summarized them with Cell C
# It will only produce figures if you ran the population sims
# with the -hi flag
# Figures saved in 'hbstats_at_times' folder
# Results likely not very good unless you have run many pop sims

# This cell plots some binned stats for the (AA or full model)
# population sims (binned by effect size, or initial MAF)
from plot_class import PlotClass


poses = ['both','pos','neg']
#Most interesting to look at aligned and opposing combined
poses = ['both']
bins = ['_efs_bins','_fbins']

statsplotall = ['frozen_d2ax_scaled_per_mut_input',
                'frozen_d2ax_over_shift',
                'frozen_nm_2ax_over_shift',
                'frozen_nm_2ax_scaled_per_mut_input',
                'frozen_numseg_per_mut_input',
                'frozen_nm_and_standing_d2ax_scaled_per_mut_input']

units = False

maxt = 10000

for data_class_param_dict in my_sum_and_save.param_dict_list():
    myparamdictkey = my_sum_and_save.turn_dict_into_dict_key(data_class_param_dict)
    datafolder = my_sum_and_save.make_folder_name_for_data_class(myparamdictkey)
    current_save_figure_directory = os.path.join(save_figure_directory,datafolder)
    tmp_dclass = my_sum_and_save.get_data_class(myparamdictkey)

    lande_time = my_sum_and_save.get_lande_time(myparamdictkey)

    # we can only make these plots if we saved histogram stats for the sims
    if tmp_dclass._hbstats:
        parameter_string = tmp_dclass.make_parameter_text_to_write_file()
        current_save_figure_directory = os.path.join(current_save_figure_directory,parameter_string)
        if not os.path.exists(current_save_figure_directory):
            os.makedirs(current_save_figure_directory)
        tmp_plots = PlotClass([tmp_dclass],base_dir=current_save_figure_directory)
        tmp_dclass.set_units(units)
        for posi in poses:
            for stati in statsplotall:
                for bin in bins:
                    nami = stati+bin
                    if nami in tmp_dclass._hbstats:
                        times = [-1,lande_time]
                        tmp_plots.plot_h_bstat_times_hist(index = 0,h_bstat=nami,times = times,pos=posi)


C 0.6840765649154079


In [7]:
# Cell K
# First run Cell H to set up sum_and_saves for plotting
#You also must have run population sims, with cell B
# and summarized them with Cell C
# It will only produce figures if you ran the population sims
# with the -hi flag
# Figures saved in 'hbstats' folder
# Results likely not very good unless you have run many pop sims

# This cell plots some binned stats for the full population sims
# (binned by effect size, or initial minor allele frequency)
from plot_class import PlotClass

poses = ['both','pos','neg']
bins = ['_efs_bins','_fbins']

statsplotall = ['frozen_d2ax_scaled_per_mut_input',
                'frozen_d2ax_over_shift','frozen_nm_2ax_scaled_per_mut_input','frozen_numseg_per_mut_input']

units = False

maxt = 10000

for data_class_param_dict in my_sum_and_save.param_dict_list():
    myparamdictkey = my_sum_and_save.turn_dict_into_dict_key(data_class_param_dict)
    datafolder = my_sum_and_save.make_folder_name_for_data_class(myparamdictkey)
    current_save_figure_directory = os.path.join(save_figure_directory,datafolder)
    tmp_dclass = my_sum_and_save.get_data_class(myparamdictkey)

    # we can only make these plots if we saved histogram stats for the sims
    if tmp_dclass._hbstats:
        parameter_string = tmp_dclass.make_parameter_text_to_write_file()
        current_save_figure_directory = os.path.join(current_save_figure_directory,parameter_string)
        if not os.path.exists(current_save_figure_directory):
            os.makedirs(current_save_figure_directory)
        tmp_dclass.set_units(units)
        tmp_plots = PlotClass([tmp_dclass],base_dir=current_save_figure_directory)

        print(tmp_dclass._hbstats)

        for posi in poses:
            for stati in statsplotall:
                for bin in bins:
                    nami = stati+bin
                    if nami in tmp_dclass._hbstats:
                        tmp_plots.plot_h_bstat_over_time_quick(index = 0,h_bstat=nami,domain=[0,maxt],pos=posi)



['frozen_nm_and_standing_d2ax_scaled_per_mut_input_efs_bins', 'frozen_d2ax_scaled_per_mut_input_fbins', 'frozen_d2ax_scaled_per_mut_input_efs_bins', 'frozen_nm_2ax_over_shift_efs_bins', 'frozen_d2ax_over_shift_fbins', 'frozen_numseg_per_mut_input_efs_bins', 'frozen_nm_2ax_scaled_per_mut_input_efs_bins', 'frozen_d2ax_over_shift_efs_bins', 'frozen_numseg_per_mut_input_fbins']


In [3]:
#Cell L
# This run plots results of trajectory simulations (all the
# previous ones did population sims)
# Plots for the end of the rapid phase
# saved in 'tstats_times'
# And long-term plots saved in 'ftstats'
# First run Cell H to set up sum_and_saves for plotting

sum_and_save_list = [my_sum_and_save]
# To run this cell with 'my_sum_and_save' in the
# sum_and_save_list you need to have run population sims, with cell B
# and summarized them with Cell C
#  You also need to have run trajectory sims
# cell D and summarized with cell G

sum_and_save_list = [my_sum_and_save_Land]
# To run this cell with 'my_sum_and_save_Land' in the
# sum_and_save_list you need to have run pur Lande trajectory sims
# with Cell E&F and summarized them with Cell G
from plot_class import PlotClass


sum_and_save_list = [my_sum_and_save_Land,my_sum_and_save]
#sum_and_save_list = [my_sum_and_save_Land]
#sum_and_save_list = [my_sum_and_save]

tstat_list_both = ['d2ax_scaled_per_mut_input']
tstat_list_pos_neg = ['x_per_seg_var']

final_stat_list_both = ['2a_frac_fixed_scaled_per_mut_input']
final_stat_list_pos_neg = ['frac_fixed_per_seg_var']

for a_sum_save_class in sum_and_save_list:
    for data_class_param_dict in a_sum_save_class.param_dict_list():
        myparamdictkey = a_sum_save_class.turn_dict_into_dict_key(data_class_param_dict)
        tmp_dclass = a_sum_save_class.get_data_class(myparamdictkey)
        datafolder = a_sum_save_class.make_folder_name_for_data_class(myparamdictkey)
        current_save_figure_directory = os.path.join(save_figure_directory,datafolder)
        parameter_string = tmp_dclass.make_parameter_text_to_write_file()
        current_save_figure_directory = os.path.join(current_save_figure_directory,parameter_string)
        tmp_plots = PlotClass([tmp_dclass],base_dir=current_save_figure_directory)
        print(data_class_param_dict)
        print(tmp_dclass._ftstats)
        lande_time = a_sum_save_class.get_lande_time(myparamdictkey)
        #lande_time = 1

        tmp_dclass.set_units(False)
        for stat in tstat_list_both:
            tmp_plots.paper_plot_tstats_times(tstat=stat,indexes=[0],time=lande_time,compare=True)
        for stat in tstat_list_pos_neg:
            for pos in ['pos','neg']:
                tmp_plots.paper_plot_tstats_times(tstat=stat,indexes=[0],time=lande_time,pos=pos)

        tmp_dclass.set_units(False)
        for stat in final_stat_list_both:
            tmp_plots.paper_plot_final_tstats(ftstat=stat,indexes=[0],compare=True)
        for stat in final_stat_list_pos_neg:
            for pos in ['pos','neg']:
                tmp_plots.paper_plot_final_tstats(ftstat=stat,indexes=[0],pos=pos)




{'N': 5000, 'Vs': 10000.0, 'shift_s0': 2.0, 'var_0': 400.0, 'sigma_0_del': 20.0, 'shift': 40.0}
['2a_frac_fixed_scaled_per_mut_input', 'NUM_SEGREGATING', 'frac_fixed_per_seg_var', '2a_frac_fixed_per_mut_input', 'NUM_MUTANTS']
we are in Lande
['d2ax_scaled_per_mut_input']
yes data
after _read_traj_bstats
we are in Lande
['x_per_seg_var']
yes data
after _read_traj_bstats
we are in Lande
['x_per_seg_var']
yes data
after _read_traj_bstats
yval is zero for S  100.0
yval is zero for S  56.23413251903491
yval is zero for S  100.0
yval is zero for S  56.23413251903491
yval is zero for S  100.0
yval is zero for S  31.622776601683793
yval is zero for S  56.23413251903491
{'N': 1000, 'Vs': 2000.0, 'shift_s0': 2.0, 'var_0': 162.15310178621888, 'sigma_0_del': 12.733935047196482, 'shift': 25.467870094392964}
['2a_frac_fixed_scaled_per_mut_input', 'NUM_SEGREGATING', 'frac_fixed_per_seg_var', '2a_frac_fixed_per_mut_input', 'NUM_MUTANTS']
no lande
['d2ax_scaled_per_mut_input']
yes data
no lande
['x_per